In [1]:
##### Takes a search query as input and get the vectors from the whole dataset to compare.
import spacy
import pickle
import numpy as np
from scipy import spatial
import sys
import unidecode
#from sklearn.decomposition import PCA
#QUERY  Neighbours Ids_and_Score_bool
directory='../'
argv=sys.argv
nlp = spacy.load("fr_core_news_lg")
pca = pickle.load(open(directory+'models/pca_30.pkl','rb'))
pca_space= np.load(directory+'models/vectors_pca_30.npy', allow_pickle=True)
id_table=list(np.load(directory+'../data/id_table.npy', allow_pickle=True))
tree = spatial.KDTree(pca_space)
from spacy.lang.fr.stop_words import STOP_WORDS
from spacy.lang.fr import French
parser=French()
stopwords = list(STOP_WORDS)

def process_query(search_query):
    query=str(search_query).lower()
    clean_query = unidecode.unidecode(query)
    tokens=parser(clean_query)
    tokens = [ word.lower_ for word in tokens ]
    tokens = [ word for word in tokens if word not in stopwords]
    tokens = " ".join([i for i in tokens])
    return (tokens)

def query2vec(search_query):
    x=nlp(search_query).vector #spacy 300d
    y=pca.transform([x])[0] #pca 30d
    return(y)

def get_id(idx):
    dataset_id=id_table[idx]
    return(dataset_id)

def get_idx(ids):
    dataset_idx=id_table.index(ids)
    return(dataset_idx)

def id2vec(ids):
    return(pca_space[get_idx(ids)])

def neighbours(vector, n):
    n_ids=[]
    score=[]
    dist, pos=tree.query(vector, k=n)
    for j in range(len(pos)):
        n_ids.append(get_id(pos[j]))
        score.append(1-dist[j]/50) ##very approximate metric 
    return(n_ids, score)

def Search(search_query, n):
    n_ids, score=neighbours(query2vec(process_query(search_query)), n)
    #print(n_ids, score)
    return(n_ids, score)

def Similarity(ids, n):
    n_ids, score=neighbours(id2vec(ids), n)
    return(n_ids, score)

In [11]:
vecss=[1,2,3,4,5]

In [12]:
vecss[:-1]

[1, 2, 3, 4]

In [9]:
vecss.append(list(id2vec(get_id(48))))

In [21]:
n_ids, score=Search('agriculture biologique', 10)

In [23]:
for i in n_ids:
    print('Neighbour  ' + data_np[get_idx(i)]+ '\n')
    print(i)

Neighbour  agriculture biologique productions végétales surfaces par département. agriculture agriculture biologique developpement durable occupation du sol

5baaf08306e3e75ac77347e9
Neighbour  agriculture biologique productions végétales surfaces par département. agriculture agriculture biologique developpement durable occupation du sol

5b935d619ce2e71929bc02c2
Neighbour  agriculture biologique productions végétales surfaces par département. agriculture agriculture biologique developpement durable occupation du sol

59d45f70b59508043de9ba09
Neighbour  agriculture biologique productions végétales surfaces par département. agriculture agriculture biologique developpement durable occupation du sol

59d30df3a3a7291993b1eae0
Neighbour  agriculture biologique productions végétales surfaces par département. agriculture agriculture biologique developpement durable occupation du sol

59cf1a61b59508043de9b878
Neighbour  agriculture biologique productions végétales surfaces par département. agr

In [2]:
import pandas as pd
df=pd.read_csv('../../data/querys.csv', sep=',',error_bad_lines=False, encoding='latin-1') #le fichier original

In [3]:
data=pd.read_pickle("../../data/nodesc_clean.pkl")
data_np=data.to_numpy()

In [4]:
pca_space[get_idx('53ba4c07a3a729219b7bead3')]

array([-0.39525387, -1.80858754, -0.99972246, -1.79877351,  3.04200497,
       -1.07497441,  1.87230447,  0.96031802,  0.22992027,  1.68786773,
       -1.46389797,  1.89697827,  0.16721121,  1.1637145 , -1.08350189,
       -0.50725967,  0.33706491,  0.6243299 ,  0.64117906,  1.80798095,
        0.43165287, -0.00331491, -2.100112  , -0.93477975,  0.91513703,
       -0.34883433, -0.66823246, -0.02884761,  0.41350963, -0.40011595])

In [5]:
df_array=np.array(df)

In [6]:
for i in range(len(df_array)):
    n_ids, score = Similarity(df_array[i][2], 10)
    print('\n'+df_array[i][0])
    for k in range(len(n_ids)):
        print('Neighbour n°: '+ str(k) + data_np[get_idx(n_ids[k])])


siren
Neighbour n°: 0arrêté du 15 septembre délibération crpmem normandie praires et amandes oc abrog 88 . autres coquillages cantonnement crpm manche normandie
Neighbour n°: 1maisons départementales des solidarités localisation. aide sociale mds solidarite
Neighbour n°: 2stationnements particuliers ville de lorient. deplacements duree limitee emplacement reserve livraison lorient personne mobilite reduite pmr stationnement taxi transports ville de lorient
Neighbour n°: 3données essentielles des marchés publics publiés sur marches publics.info aws . commande publique donnees essentielles
Neighbour n°: 4annuaire géolocalisé. administration adresse annuaire citoyennete courriel email finances publiques gouvernement mail telephone
Neighbour n°: 5température quotidienne régionale depuis janvier . developpement durable meteorologie region territoire territoires et regions
Neighbour n°: 6dette propre au 1er janvier de l'exercice. budget dette propre finances departementales territoire
Neigh


contours départements
Neighbour n°: 0accords mets et vins. alimentation cuisine gastronomie oenologie vin
Neighbour n°: 1contours géographiques des régions – . admin express france geospatial ign region territoires
Neighbour n°: 2périmètres des secteurs de recrutements des collèges publics de gironde. college conseil departemental gironde limites administratives
Neighbour n°: 3plan départemental des itinéraires de promenades et de randonnées pdipr . nan
Neighbour n°: 4résultats du premier tour de la primaire de la belle alliance. election parti socialiste presidentielle primaire raicaux
Neighbour n°: 5collecte des archives municipales. culture loisirs sports
Neighbour n°: 6economie zae grand poitiers données métiers. activite activite economique donnees ouvertes economique economy passerelle inspire zones
Neighbour n°: 7votre commerce roubaix. activites business commercant commerces developpement economique economie emploi hebergement label zero dechet magasin pme restaurant restaurat


contours départements français
Neighbour n°: 0accords mets et vins. alimentation cuisine gastronomie oenologie vin
Neighbour n°: 1contours géographiques des régions – . admin express france geospatial ign region territoires
Neighbour n°: 2périmètres des secteurs de recrutements des collèges publics de gironde. college conseil departemental gironde limites administratives
Neighbour n°: 3plan départemental des itinéraires de promenades et de randonnées pdipr . nan
Neighbour n°: 4résultats du premier tour de la primaire de la belle alliance. election parti socialiste presidentielle primaire raicaux
Neighbour n°: 5collecte des archives municipales. culture loisirs sports
Neighbour n°: 6economie zae grand poitiers données métiers. activite activite economique donnees ouvertes economique economy passerelle inspire zones
Neighbour n°: 7votre commerce roubaix. activites business commercant commerces developpement economique economie emploi hebergement label zero dechet magasin pme restaurant 

ValueError: '53699058a3a729239d2039a3' is not in list

In [38]:
scores=[]
positions=[]
for i in range(len(df_array)):
    n_ids, score = Search(df_array[i][0], 20)
    #print(n_ids)
    #print(df_array[i][2])
    for k in range(len(n_ids)):
        if n_ids[k]==df_array[i][2]:
            score=1-k/len(n_ids)
            pos=k
            print('FOUND :' + str(df_array[i][0])+ ' in pos '+ str(k))
            break
        else:
            score=0
            pos=k
            if k==len(n_ids)-1:
                print('DIDNT FIND:'+str(df_array[i][0]))
    scores.append(score)
    positions.append(k)

DIDNT FIND:siren
DIDNT FIND:sirene
DIDNT FIND:entreprise
DIDNT FIND:entreprises
DIDNT FIND:siret
DIDNT FIND:open damir
DIDNT FIND:opendamir
DIDNT FIND:damir
FOUND :contours départements in pos 6
DIDNT FIND:emissions polluantes
DIDNT FIND:géofla départements
FOUND :effectifs police municipale in pos 10
FOUND :marchés public bourgogne in pos 2
FOUND :Liste gares SNCF in pos 0
DIDNT FIND:contours départements français
FOUND :loi de finance 2016 in pos 9
DIDNT FIND:lolf 2016
DIDNT FIND:formations pas de calais
DIDNT FIND:accidents de la circulation
DIDNT FIND:accidents de la route
FOUND :risque de décès un an après accident in pos 2
DIDNT FIND:COG
FOUND :code officiel géographique in pos 0
DIDNT FIND:contour commune
DIDNT FIND:contours communes
DIDNT FIND:contour communes
FOUND :code postal in pos 7
DIDNT FIND:codes postaux
DIDNT FIND:prénoms
DIDNT FIND:association
DIDNT FIND:associations
DIDNT FIND:RNA
DIDNT FIND:nan
DIDNT FIND:répertoire des associations
DIDNT FIND:répertoire national de

In [52]:
df

,query,params,expected
0,siren,NaN,5b7ffc618b4c4169d30727e0
1,sirene,NaN,5b7ffc618b4c4169d30727e0
2,entreprise,NaN,5b7ffc618b4c4169d30727e0
3,entreprises,NaN,5b7ffc618b4c4169d30727e0
4,siret,NaN,5b7ffc618b4c4169d30727e0
5,open damir,NaN,54de1e8fc751df388646738b
6,opendamir,NaN,54de1e8fc751df388646738b
7,damir,NaN,54de1e8fc751df388646738b
8,contours départements,NaN,536991b0a3a729239d203d13
9,emissions polluantes,NaN,53ba4c07a3a729219b7bead3


In [10]:
scores

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0.94,
 0,
 0,
 0.9,
 0.98,
 1.0,
 0,
 0.91,
 0,
 0,
 0.55,
 0,
 0.98,
 0,
 1.0,
 0,
 0,
 0,
 0.9299999999999999,
 0,
 0,
 0.61,
 0,
 0,
 0,
 0,
 0.8,
 0,
 0,
 0,
 0,
 0,
 0.8200000000000001,
 0]

In [48]:
df=df.drop(17)

In [17]:
get_id(40989)

'53699489a3a729239d204488'